# Persistance

ressource: https://langchain-ai.github.io/langgraph/concepts/persistence

## Checkpoints


In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import Annotated
from typing_extensions import TypedDict
from operator import add
from pprint import pprint

In [ ]:
class State(TypedDict):
    foo: str
    bar: Annotated[list[str], add]

In [ ]:
def node_a(state: State):
    return {"foo": "a", "bar": ["a"]}

In [ ]:
def node_b(state: State):
    return {"foo": "b", "bar": ["b"]}

In [ ]:
workflow = StateGraph(State)

In [ ]:
workflow.add_node(node=node_a)
workflow.add_node(node=node_b)

In [ ]:
workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", END)

In [ ]:
checkpointer = InMemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

In [ ]:
config = {"configurable": {"thread_id": "1"}}
graph.invoke({"foo": ""}, config)

## Get state

In [ ]:
# Get the latest state snapshot
config = {"configurable": {"thread_id": "1"}}
pprint(graph.get_state(config=config))

In [ ]:
# Get a state snapshot for a specific checkpoint_id
config = {"configurable": {"thread_id": "1", "checkpoint_id": "1f044dd5-8725-69d4-8002-7134a86a0d7f"}}
graph.get_state(config=config)

## Get state history & replay

In [ ]:
config = {"configurable": {"thread_id": "1"}}
pprint(list(graph.get_state_history(config=config)))

In [ ]:
config = {"configurable": {"thread_id": "1", "checkpoint_id": "1f044dd5-8712-6033-8000-2161a17ab1dc"}}
graph.invoke({"foo": "z"}, config=config)

In [ ]:
pprint(list(graph.get_state_history(config={"configurable": {"thread_id": "1"}})))

In [ ]:
config = {"configurable": {"thread_id": "1", "checkpoint_id": "1f044dd5-8712-6033-8000-2161a17ab1dc"}}
graph.invoke({"bar": ["42"]}, config=config)

In [ ]:
pprint(list(graph.get_state_history(config={"configurable": {"thread_id": "1"}})))

## Memory store

#### Basic usage

In [4]:
from langgraph.store.memory import InMemoryStore
import uuid

In [2]:
in_memory_store = InMemoryStore()

In [3]:
user_id = "1"
namespace_for_memory = (user_id, "memories")

In [5]:
memory_id = str(uuid.uuid4())
memory = {"food_preference": "I like pizza"}
in_memory_store.put(namespace=namespace_for_memory, key=memory_id, value=memory)

In [6]:
memories = in_memory_store.search(namespace_for_memory)
memories[-1].dict()

{'namespace': ['1', 'memories'],
 'key': '0be9a0a1-899a-4c54-b580-7b86a59ac5c0',
 'value': {'food_preference': 'I like pizza'},
 'created_at': '2025-06-10T19:10:10.313727+00:00',
 'updated_at': '2025-06-10T19:10:10.313727+00:00',
 'score': None}

In [8]:
memories

[Item(namespace=['1', 'memories'], key='0be9a0a1-899a-4c54-b580-7b86a59ac5c0', value={'food_preference': 'I like pizza'}, created_at='2025-06-10T19:10:10.313727+00:00', updated_at='2025-06-10T19:10:10.313727+00:00', score=None)]

#### Semantic search

In [1]:
# from langchain.embeddings import init_embeddings
from langgraph.store.memory import InMemoryStore
from langchain_ollama.embeddings import OllamaEmbeddings
import uuid

In [2]:
user_id = "1"
namespace_for_memory = (user_id, "memories")

In [3]:
store = InMemoryStore(
    index={
        "embed": OllamaEmbeddings(model="granite-embedding:278m"),  # Embedding provider
        "dims": 1536,  # Embedding dimensions
        "fields": ["food_preference","$"]  # Fields to embed
    }
)

In [4]:
# Find memories about food preferences
# (This can be done after putting memories into the store)
memories = store.search(
    namespace_for_memory,
    query="What does the user like to eat?",
    limit=3  # Return top 3 matches
)

In [5]:
memories

[]

In [6]:
# Store with specific fields to embed
store.put(
    namespace=namespace_for_memory,
    key=str(uuid.uuid4()),
    value={
        "food_preference": "I love Italian cuisine",
        "context": "Discussing dinner plans"
    },
    index=["food_preference"]  # Only embed "food_preference" field
)

In [7]:
# Store without embedding (still rerievable, bbut not searchable)
store.put(
    namespace=namespace_for_memory,
    key=str(uuid.uuid4()),
    value={
        "system_info": "Last updated: 2025-04-01"
    },
    index=False,
)

In [8]:
memories = store.search(
    namespace_for_memory,
    query="What does the user like to eat?",
    limit=3  # Return top 3 matches
)

In [9]:
memories

[Item(namespace=['1', 'memories'], key='bbd4e492-d3b6-4341-8c14-11530f0a347c', value={'food_preference': 'I love Italian cuisine', 'context': 'Discussing dinner plans'}, created_at='2025-06-13T22:10:37.573013+00:00', updated_at='2025-06-13T22:10:37.573013+00:00', score=0.6458198803706252),
 Item(namespace=['1', 'memories'], key='80a7f034-1553-42e9-bbf6-11543223bbdf', value={'system_info': 'Last updated: 2025-04-01'}, created_at='2025-06-13T22:10:41.049718+00:00', updated_at='2025-06-13T22:10:41.049718+00:00', score=None)]

In [10]:
memory_id = str(uuid.uuid4())
memory = {"food_preference": "I like pizza", "context": "Discussing food preference"}
store.put(namespace=namespace_for_memory, key=memory_id, value=memory, index=["food_preference"])

In [11]:
store.search(
    namespace_for_memory,
    query="What does the user like to eat?",
    limit=3  # Return top 3 matches
)

[Item(namespace=['1', 'memories'], key='bbd4e492-d3b6-4341-8c14-11530f0a347c', value={'food_preference': 'I love Italian cuisine', 'context': 'Discussing dinner plans'}, created_at='2025-06-13T22:10:37.573013+00:00', updated_at='2025-06-13T22:10:37.573013+00:00', score=0.6458198803706255),
 Item(namespace=['1', 'memories'], key='716823b0-e70f-4964-b9ab-b3ef6c4aaac0', value={'food_preference': 'I like pizza', 'context': 'Discussing food preference'}, created_at='2025-06-13T22:10:50.970458+00:00', updated_at='2025-06-13T22:10:50.970458+00:00', score=0.624045188728224),
 Item(namespace=['1', 'memories'], key='80a7f034-1553-42e9-bbf6-11543223bbdf', value={'system_info': 'Last updated: 2025-04-01'}, created_at='2025-06-13T22:10:41.049718+00:00', updated_at='2025-06-13T22:10:41.049718+00:00', score=None)]

#### Using in LangChain

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

In [13]:
# We need this because we want to enable threads (conversations)
checkpointer = InMemorySaver()

In [ ]:
# Define the graph
